In [18]:
# Basic libs
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


# Custom libs
from MHealthDataset import MHealthDataset
from Net import Net
training_data = MHealthDataset(data_dir="data/data.csv",labels_dir="data/labels.csv")
test_data = MHealthDataset(data_dir="data/data.csv",labels_dir="data/labels.csv")

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model = Net(60)
print(model)

Using cuda device
Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=60, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=13, bias=True)
  )
)


# Training

In [20]:
learning_rate = 1e-3
batch_size = 30

In [36]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            aa = (pred.argmax(1) == y.argmax(1) ).type(torch.float).sum().item()
            correct += aa

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [37]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 1.949879  [    0/34318]
loss: 0.634038  [ 6400/34318]
loss: 0.532386  [12800/34318]
loss: 0.742001  [19200/34318]
loss: 1.317618  [25600/34318]
loss: 1.454551  [32000/34318]
Test Error: 
 Accuracy: 47.1%, Avg loss: 1.319047 

Epoch 2
-------------------------------
loss: 1.948334  [    0/34318]
loss: 0.608279  [ 6400/34318]
loss: 0.511896  [12800/34318]
loss: 0.725903  [19200/34318]
loss: 1.301530  [25600/34318]
loss: 1.418373  [32000/34318]
Test Error: 
 Accuracy: 47.3%, Avg loss: 1.307770 

Epoch 3
-------------------------------
loss: 1.955853  [    0/34318]
loss: 0.583310  [ 6400/34318]
loss: 0.496933  [12800/34318]
loss: 0.725875  [19200/34318]
loss: 1.270948  [25600/34318]
loss: 1.373815  [32000/34318]
Test Error: 
 Accuracy: 47.5%, Avg loss: 1.295937 

Epoch 4
-------------------------------
loss: 1.956322  [    0/34318]
loss: 0.562587  [ 6400/34318]
loss: 0.487214  [12800/34318]
loss: 0.725405  [19200/34318]
loss: 1.238751  [25600/3